# LongChain: Memory

## Outline
* ConversationBufferMemory
* ConversionBufferWindowsMemory
* ConversionTokenBufferMemory
* ConversationSummaryMemory

In [22]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import warnings
warnings.filterwarnings('ignore')

In [23]:
# Account for deprecation LLM model
import datetime
current_date = datetime.datetime.now().date()  # Get the current date

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## ConversationBufferMemory

In [24]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [25]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain (
    llm = llm,
    memory = memory,
    # verbose = False
    verbose = True  # To see what langchain is actually doing
)

In [26]:
conversation.predict(input="Hi, my name is Hassane")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Hassane
AI:

> Finished chain.


"Hello Hassane! It's nice to meet you. How can I assist you today?"

In [27]:
conversation.predict(input="What is 1+1")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Hassane
AI: Hello Hassane! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

In [28]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Hassane
AI: Hello Hassane! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Hassane. Is there anything else you would like to know or talk about?'

As we can in the above cells, the conversation history/memory is getting longer and longer

In [29]:
print(memory.buffer)

Human: Hi, my name is Hassane
AI: Hello Hassane! It's nice to meet you. How can I assist you today?
Human: What is 1+1
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Hassane. Is there anything else you would like to know or talk about?


In [30]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Hassane\nAI: Hello Hassane! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1\nAI: 1 + 1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Hassane. Is there anything else you would like to know or talk about?"}

In [31]:
memory = ConversationBufferMemory()

In [32]:
# Add new things to the memory
memory.save_context({"input": "Hi"}, {"outputs": "What\'s up"})

In [33]:
print(memory.buffer)

Human: Hi
AI: What's up


In [34]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [35]:
memory.save_context({"input": "Not much, just hangin"}, {"output": "Cool"})

In [36]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hangin\nAI: Cool"}

## ConversationBufferWindowMemory

In [37]:
from langchain.memory import ConversationBufferWindowMemory

In [38]:
memory = ConversationBufferWindowMemory(k=1)

In [39]:
memory.save_context({"input": "Hi"}, {"outputs": "What\'s up"})
memory.save_context({"input": "Not much, just hangin"}, {"output": "Cool"})

In [40]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hangin\nAI: Cool'}

In [41]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain (
    llm = llm,
    memory = memory,
    verbose = True
)

In [42]:
conversation.predict(input="Hi, my name is Hassane")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Hassane
AI:

> Finished chain.


"Hello Hassane! It's nice to meet you. How can I assist you today?"

In [43]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Hassane
AI: Hello Hassane! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

In [44]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to ask?"

## ConversionTokenBufferMemory

With a conversational token buffer memory, the memory will limit the number of tokens saved

In [45]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [46]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

In [47]:
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},  {"output": "Charming!"})

In [48]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversatioSummaryMemory

In [49]:
from langchain.memory import ConversationSummaryBufferMemory

In [50]:
# Create a long string
schedule = "There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand \
the latest in AI. Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=300)

In [51]:

memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"})

In [52]:
memory.load_memory_variables({})

{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo."}

In [53]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [54]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is on the schedule today?
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'I would recommend showcasing the latest Language Model demo, as it is a cutting-edge technology that your customer will likely be very interested in. It demonstrates the power and capabilities of our AI technology in a tangible way. Additionally, you could also show some real-world examples of how our AI has helped other customers improve their processes and achieve their goals.'

In [55]:
memory.load_memory_variables({})

{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.\nHuman: What would be a good demo to show?\nAI: I would recommend showcasing the latest Language Model demo, as it is a cutting-edge technology that your customer will likely be very interested in. It demonstrates the power and capabilities of our AI technology in a tangible way. Additionally, you could also show some real-world examples of how our AI has helped other customers improve their processes and achieve their goals."}